In [ ]:
### Q10
import pandas as pd
import numpy as np
data = """Event ID, Case ID, Activity, Timestamp
e1 x1 a 1
e2 x1 c 4
e3 x2 a 1
e4 x2 b 2
e5 x1 b 2
e6 x2 c 4
e7 x1 b 3"""
data = data.splitlines()
columns = data[0].split(", ")
data = [i.split() for i in data[1:]]
df = pd.DataFrame(data, columns=columns)
df = df.astype({"Timestamp": int})
display(df)

In [ ]:
# (a)
df_a = df.copy()

def map_step(entry:tuple) -> list[int, list[tuple]]:
    return entry[2], entry

def reduce_step(key, values:list[tuple]):
    # Index row is also present so activity is at 3 and timestamp at 4
    activities = [(row[3],row[4]) for row in values]
    activities = sorted(activities,key=lambda x:int(x[1]))
    return key,(x[0] for x in activities)

def map_reduce(df):
    map_dict = dict()
    for row in df.itertuples():
        key,values = map_step(row)
        if key in map_dict:
            map_dict[key].append(values)
        else:
            map_dict[key] = [values]
    results = list()
    for key in map_dict:
        results.append(reduce_step(key,map_dict[key]))
    return results

for result in map_reduce(df_a):
    print(result[0],list(result[1]))

In [ ]:
# (b) Come on this is barely Big Data related it's just an algorithm (of many) in the BigData area not a central concept
# !!!! That being said what follows is ALL WRONG, see next cell for proper implementation
# Cluster assignment happens based on distance
# so all points get assigned to c1 in the first step
# New centroids are computed as 
# sum(w_c*c,other_points)/(w_c+len(other_points))
# I don't actually know if alpha*w_c gets executed before or after
# but I assume after.
alpha = 0.5
c1 = np.array((0,0))
w_c1 = 1.0
c2 = np.array((10,10))
w_c2 = 1.0
def step(new_instances,alpha,c1,w_c1,c2,w_c2):
    cluster1 = []
    cluster2 = []
    for point in new_instances:
        if np.linalg.norm(c1-point) > np.linalg.norm(c2-point):
            cluster2.append(point)
        else:
            cluster1.append(point)
    new_c1 = (sum(cluster1)+w_c1*c1)/(len(cluster1)+w_c1)
    new_c2 = (sum(cluster2)+w_c2*c2)/(len(cluster2)+w_c2)
    return new_c1,alpha*w_c1,new_c2,alpha*w_c2

new_instances = np.array(
    [
        (3,3),
        (-3,3),
        (1,3),
        (-1,0)
    ]
)
c1,w_c1,c2,w_c2 = step(new_instances,alpha,c1,w_c1,c2,w_c2)
display(c1,w_c1,c2,w_c2)

c1,w_c1,c2,w_c2 = step(new_instances,alpha,c1,w_c1,c2,w_c2)
display(c1,w_c1,c2,w_c2)



In [ ]:
# (b) 
alpha = 0.5
c1 = np.array((0,0))
w_c1 = 1.0
c2 = np.array((10,10))
w_c2 = 1.0

threshold = 0.8

new_instances = np.array(
    [
        (3,3),
        (-3,3),
        (1,3),
        (-1,0)
    ]
)

def step(new_instances,alpha,c1,w_c1,c2,w_c2):
    cluster1 = []
    cluster2 = []
    for point in new_instances:
        if np.linalg.norm(c1-point) > np.linalg.norm(c2-point):
            cluster2.append(point)
        else:
            cluster1.append(point)
    new_c1 = (sum(cluster1)*len(cluster1)+alpha*w_c1*c1)/(len(cluster1)+(alpha*w_c1))
    new_c2 = (sum(cluster2)*len(cluster2)+alpha*w_c2*c2)/(len(cluster2)+(alpha*w_c2))
    return new_c1,alpha*w_c1+len(cluster1),new_c2,alpha*w_c2+len(cluster2)


c1,w_c1,c2,w_c2 = step(new_instances,alpha,c1,w_c1,c2,w_c2)
print(f"New Centroid 1 at ({c1[0]:.2f},({c1[1]:.2f}) with weight {w_c1:.2f}")
print(f"New Centroid 2 at ({c2[0]:.2f},({c2[1]:.2f}) with weight {w_c2:.2f}")

if w_c1 < threshold:
    print("Cluster 1 is subcritical. Kick it out.")

if w_c2 < threshold:
    print("Cluster 2 is subcritical. Kick it out.")